In [899]:
import pandas as pd
from copy import deepcopy
import copy
import math as math
import numpy as np
from matplotlib import pyplot as plt
import time, os, sys, json,pickle
from math import exp
import itertools
from smart_open import open

# Similarity and normalization packages
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.preprocessing import MinMaxScaler, minmax_scale
import scipy
from scipy import stats
import scipy.spatial
# Machine learninig packages
from sklearn import preprocessing
from sklearn import linear_model, ensemble, metrics
from sklearn.metrics import mean_squared_error, r2_score
from lifelines.utils import concordance_index
from sklearn.metrics import *

# for protein embedding
import biovec

# for drugs embedding
import tensorflow as tf
from tensorflow import keras

# deepchem library
import deepchem as dc
from deepchem.feat import Featurizer
from deepchem.models.optimizers import ExponentialDecay, Adam
from deepchem.metrics import to_one_hot
from deepchem.trans import undo_transforms
from deepchem.models.graph_models import GraphConvModel, L2Loss,Dense,  Reshape,Dropout 
import networkx

import xgboost as xgb

# Import my files
from training_functions import *
from pathScores_functions import *
from evaluation import *

In [240]:
# Load the dataset
all_DTA_info = pd.read_csv('PDBBind_Refined/All_PDBbind_info.csv')
all_DTA_info.head(4)

,LigandID,pdbID,Affinity,Year,SMILES,Seq
0,MLY,2r58,2.000000,2007,C[NH+](C)CCCCC([NH3+])C(=O)O,AFDWDAYLEETGSEAAPAKCFKQAQNPPNNDFKIGMKLEALDPRNV...
1,PRP,3c2f,1.995679,2008,O=P(O)(O)OCC1OC(OP(=O)(O)OP(=O)(O)O)C(O)C1O,PVYEHLLPVNGAWRQDVTNWLSEDVPSFDFGGYVVGSDLKEANLYC...
2,3HP,3pce,2.000000,1998,O=C(O)Cc1cccc(O)c1,PIELLPETPSQTAGPYVHIGLALEAAGNPTRDQEIWNRLAKPDAPG...
3,TDR,4qsu,2.000000,2014,CC1C=NC(=O)NC1=O,SMQEEDTFRELRIFLRNVTHRLAIDKRFRVFTKPVDPDEVPDYVTV...


# Ligand-Protein complexes Preprocessig

In [351]:
# Ligand Preproess
DP_aff = all_DTA_info[["LigandID", "pdbID","Affinity"]]
print(DP_aff.shape)
edgeList = DP_aff.values.tolist()
edgeList = np.array(edgeList)

ligand_no = DP_aff['LigandID'].nunique()
protein_no = DP_aff['pdbID'].nunique()

allD = DP_aff['LigandID'].unique()
allP = DP_aff['pdbID'].unique()

# create indexing-dictionaries for drugs (Ligands) and for targets (Proteins)
DrID = dict([(d, i) for i, d in enumerate(allD)])
PrID = dict([(t, i) for i, t in enumerate(allP)])

ligand_no, protein_no

(2326, 3047)

In [337]:
ligand_un_df = all_DTA_info.copy()
print(ligand_un_df.shape)
ligand_un_df = ligand_un_df.drop_duplicates(subset=['LigandID'])
print(ligand_un_df.shape)
ligand_un_df.head(3)

(3047, 5)
(2326, 5)


,pdbID,LigandID,Affinity,SMILES,Seq
0,2r58,MLY,2.000000,C[NH+](C)CCCCC([NH3+])C(=O)O,AFDWDAYLEETGSEAAPAKCFKQAQNPPNNDFKIGMKLEALDPRNV...
1,3c2f,PRP,1.995679,O=P(O)(O)OCC1OC(OP(=O)(O)OP(=O)(O)O)C(O)C1O,PVYEHLLPVNGAWRQDVTNWLSEDVPSFDFGGYVVGSDLKEANLYC...
2,3pce,3HP,2.000000,O=C(O)Cc1cccc(O)c1,PIELLPETPSQTAGPYVHIGLALEAAGNPTRDQEIWNRLAKPDAPG...


In [900]:
DP_Mat,label_row_inds,label_col_inds = edgelist_to_adjMat(edgeList,DrID,PrID)
DP_Mat.shape, len(label_row_inds),len(label_col_inds)
DP_Mat

array([[ 2.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  1.99567863,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  2.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ..., 11.82390874,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        11.85387196,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , 11.92081875]])

In [776]:
# New affinity score for graph G..
aff_exp = np.exp((-1)*DP_Mat)
aff_exp

array([[1.35335283e-01, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 1.35921383e-01, 1.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 1.00000000e+00, 1.35335283e-01, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       ...,
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        7.32726130e-06, 1.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 7.11096951e-06, 1.00000000e+00],
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 6.65049843e-06]])

# Learn Embeddings for ligand and proteins
## 1) Learn amino-acid sequnce embeddings using ProtVec trined model

In [5]:
# import trained model
pv = biovec.models.load_protvec('swissprot-reviewed-protvec.model')

Embed = {}
seqEmbed = []
sumEmbed = []
avgEmbed = []
allEmbed = []

for i in range(all_DT_df.shape[0]):
    embedding = pv.to_vecs(all_DT_df['Seq'][i])
    # Dictionary to get embedding by target ID
    Embed[all_DT_df['pdbID'][i]] = embedding
    
    allem = list(itertools.chain(embedding[0],embedding[1],embedding[2]))
    #concatenate 3d
    allEmbed.append(allem)
    #list of embedding 3D
    seqEmbed.append(embedding)
    
    # 1D embedding by taking sum of 3 ists
    sumE = embedding[0]+embedding[1]+embedding[2]
    sumEmbed.append(sumE)
    
    #1D embedding by taking the average
    avgEmbed.append(sumE/3)
    
avgEmbed = np.array(avgEmbed)
sumEmbed = np.array(sumEmbed)

/home/thafarma/anaconda3/lib/python3.8/site-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  warnings.warn(


## 2) Learn SMILES embeddings using seq2seq fingerprint
### a) Prepare SMILES, tokens, and read more smiles data to train the model

In [7]:
## Read 
tasks, datasets, transformers = dc.molnet.load_muv()
train_dataset, valid_dataset, test_dataset = datasets
train_smiles = train_dataset.ids
valid_smiles = valid_dataset.ids
train_smiles_list = list(train_smiles)
valid_smiles_list = list(valid_smiles)

In [8]:
###.. GET DAVIS SMILES
# reading the smiles from the file  (Drugs SMILES)
with open('Input/Davis/ligands_can.txt') as f: 
    dr_data = f.read() 
# reconstructing the data as a dictionary 
dr_dv_dic = json.loads(dr_data) 
dr_dv_keys = list(dr_dv_dic.keys())
dr_dv_Sm =  list(dr_dv_dic.values())

#---------------------------------------------
###.. GET KIBA SMILES
# Kiba SMILES
with open('Input/Kiba/ligands_can.txt') as f: 
    dr_kiba = f.read() 
# reconstructing the data as a dictionary 
dr_kiba_dic = json.loads(dr_kiba) 
#print(dr_sm_dic)
dr_keys_kiba = list(dr_kiba_dic.keys())
dr_kiba_Sm =  list(dr_kiba_dic.values())


In [9]:
dr_pdb_Sm = list(all_DT_df['SMILES'])
allSM = dr_dv_Sm + dr_kiba_Sm + train_smiles_list + valid_smiles_list + dr_pdb_Sm
all_smiles = np.array(allSM)
len(dr_pdb_Sm), len(allSM)

(3047, 89004)

In [10]:
allTokens = set()
for s in all_smiles:
    allTokens = allTokens.union(set(c for c in s))
allTokens = sorted(list(allTokens))

max_length = max(len(s) for s in all_smiles)
len(allTokens)

35

### b). Creat seq2seq model to learn embeddings for all ligands

In [12]:
## Train Seq2Seq model
batch_size = 100
model = dc.models.SeqToSeq(allTokens,
                           allTokens,
                           max_length,
                           encoder_layers=2,
                           decoder_layers=2,
                           variational=True,
                           embedding_dimension=128,
                           model_dir='fingerprint')



batches_per_epoch = len(all_smiles)/batch_size
model.optimizer=Adam(learning_rate=ExponentialDecay(0.005, 0.9, batches_per_epoch))

In [13]:
def generate_sequences(epochs):
    for i in range(epochs):
        for s in all_smiles:
            yield (s, s)
model.fit_sequences(generate_sequences(40))

In [777]:
pdb_sm_unique = np.array(list(ligand_un_df['SMILES']))
pdb_sm_unique.shape

(2326,)

In [906]:
# Generate embeddings
pdb_sm_EMBED = model.predict_embeddings(dr_pdb_Sm)
pdb_sm_EMBED=np.array(pdb_sm_EMBED)

pdb_sm_EMBED_u = model.predict_embeddings(pdb_sm_unique)
pdb_sm_EMBED_u =np.array(pdb_sm_EMBED_u)

pdb_sm_EMBED.shape, pdb_sm_EMBED_u.shape, sumEmbed.shape 

((3047, 128), (2326, 128), (3047, 100))

In [919]:
file = "trained_seq2seq_SMILES_model.pickle.dat"
# save model to file
# pickle.dump(model, open(file, 'wb'))

In [907]:
# Write Sequence Embeddings and SMILES Embeddings into file
################################################################
Pr_EMBED_df = pd.DataFrame.from_dict(sumEmbed)
Pr_EMBED_df.to_csv('PDBBind_Refined/EMBED/Pr_ProtVec_EMBED.txt',sep=' ',index=None,header=None)

pdb_EMBED_df = pd.DataFrame.from_dict(pdb_sm_EMBED)
pdb_EMBED_df.to_csv('PDBBind_Refined/EMBED/Dr_seq2seq_EMBED.txt',sep=' ',index=None,header=None)

pdb_EMBED_df_u = pd.DataFrame.from_dict(pdb_sm_EMBED_u)
pdb_EMBED_df_u.to_csv('PDBBind_Refined/EMBED/Dr_seq2seq_EMBED_u.txt',sep=' ',index=None,header=None)

## Prepare the data to construct the graph G(V,E)

In [372]:
# #Clculate cosine similarity for ach pair of proteins and each pair of drugs (ligands)

DD_Sim_sm = Cosine_Similarity(pdb_EMBED_df)
PP_Sim_sq = Cosine_Similarity(sumEmbed)

# #normalize simiarities to be in positive range [0,1]
DD_Sim_sm = normalizedMatrix(DD_Sim_sm)
PP_Sim_sq = normalizedMatrix(PP_Sim_sq)

In [783]:
# Pr_SimM = keep_sim_threshold(Pr_SimM ,0.04)
# Dr_SimM = keep_sim_threshold(Dr_SimM ,0.35)
DD_Sim_sm.shape, PP_Sim_sq.shape 

((2326, 2326), (3047, 3047))

In [502]:
XPair = all_DTA_info[["LigandID", "pdbID"]]
XPair = XPair.to_numpy()

Y = np.array(all_DTA_info['Affinity'])

XPair.shape, Y.shape

((3047, 2), (3047,))

### *Create the training, validation and test sets (setting-2)
#### (time-base split using the publication years).

In [455]:
all_DTA_info.head(3)

,LigandID,pdbID,Affinity,Year,SMILES,Seq
0,MLY,2r58,2.000000,2007,C[NH+](C)CCCCC([NH3+])C(=O)O,AFDWDAYLEETGSEAAPAKCFKQAQNPPNNDFKIGMKLEALDPRNV...
1,PRP,3c2f,1.995679,2008,O=P(O)(O)OCC1OC(OP(=O)(O)OP(=O)(O)O)C(O)C1O,PVYEHLLPVNGAWRQDVTNWLSEDVPSFDFGGYVVGSDLKEANLYC...
2,3HP,3pce,2.000000,1998,O=C(O)Cc1cccc(O)c1,PIELLPETPSQTAGPYVHIGLALEAAGNPTRDQEIWNRLAKPDAPG...


In [136]:
## Setting --2

test_index = []
train_index = []
valid_index = []
for i in range (0,all_DTA_info.shape[0]):
    if(all_DTA_info['Year'][i] <=2011):
        train_index.append(i)
   
    elif(all_DTA_info['Year'][i]==2012):
        valid_index.append(i)
                
    else:
        test_index.append(i)

In [795]:
len(test_index), len(valid_index), len(train_index)

(547, 312, 2188)

## 1- Affinity2Vec Embed Model

In [800]:
FV_targets_sq = np.array(sumEmbed, dtype = float)
FV_drugs_sq = np.array(pdb_sm_EMBED, dtype = float)

concatenateFV = []
class_labels = []
lab = []
for i in range (all_DTA_info.shape[0]):
#     print(i)
    features = list(FV_drugs_sq[i]) + list(FV_targets_sq[i])
    concatenateFV.append(features)
    label = all_DTA_info['Affinity'][i]
    lab.append(label)

In [901]:
X = np.array(concatenateFV)
YY = np.array(lab)
print('ConcatFV shape',X.shape)

ConcatFV shape (3047, 228)


In [902]:
xg_reg= xgb.XGBRegressor(booster = 'gbtree', objective ='reg:squarederror', 
        eval_metric = 'rmse', colsample_bytree = 0.8, learning_rate = 0.01, max_depth = 15,
        scale_pos_weight = 1, gamma=0,alpha = 5,n_estimators = 655, tree_method='gpu_hist',
        min_child_weight =6,seed=10, gpu_id=1, n_jobs=-1) 

In [903]:
# normalize the feature vector
min_max_scaler = MinMaxScaler()
X_train = min_max_scaler.fit(X[train_index])
X_train_transform = min_max_scaler.transform(X[train_index])

# Train the model using the training sets
xg_reg.fit(X_train_transform, Y[train_index])

XGBRegressor(alpha=5, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eval_metric='rmse',
             gamma=0, gpu_id=1, importance_type='gain',
             interaction_constraints='', learning_rate=0.01, max_delta_step=0,
             max_depth=15, min_child_weight=6, missing=nan,
             monotone_constraints='(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)',
             n_estimators=655, n_jobs=-1, num_parallel_tree=1, random_state=10,
             reg_alpha=5, reg_lambda=1, scale_pos_weight=1, seed=10,
             subsample=1, tree_method='gpu_hist', validate_parameters=1,
             verbosity=None)

In [904]:
#######################Validation SET Evaluation######################### 
X_valid_transform = min_max_scaler.transform(X[valid_index])
predictedY= xg_reg.predict(X_valid_transform)

print("----------------- *EVALUATION using Validation set* -----------------")
# Evaluation Metrics (RMSE, CI, and AUPR)
print('Mean Squared Error: %.3f' % mean_squared_error(Y[valid_index], predictedY))
print('Concordance index: %3f' % get_cindex(Y[valid_index], predictedY))
print('aupr: %.3f' % average_AUPRC(Y[valid_index], predictedY))


----------------- *EVALUATION using Validation set* -----------------
Mean Squared Error: 2.146
Concordance index: 0.733445
aupr: 0.802


In [905]:
########################### Test SET Evaluation

print("----------------- *Test Set EVALUATION* -----------------")
X_test_transform = min_max_scaler.transform(X[test_index])
# Make predictions using the testing set
predictedY_test= xg_reg.predict(X_test_transform)

# Evaluation Metrics (RMSE, CI, and AUPR)
print('Mean Squared Error: %.3f' % mean_squared_error(Y[test_index], predictedY_test))
print('Concordance index: %3f' % get_cindex(Y[test_index], predictedY_test))
print('aupr: %.3f' % average_AUPRC(Y[test_index], predictedY_test))

----------------- *Test Set EVALUATION* -----------------
Mean Squared Error: 2.254
Concordance index: 0.727442
aupr: 0.791


## 2- Affinity2Vec Path-scors model

In [528]:
#first thing with affinity train to remove all edges in test set
train_aff_M = Mask_test_index(test_index, XPair, aff_exp, DrID, PrID)

In [529]:
# Generate all featres from the matrix multiplication under each path strucutre..

sumDDD, maxDDD = DDD_TTT_sim(DD_Sim_sm)
sumTTT, maxTTT= DDD_TTT_sim(PP_Sim_sq)

sumDDT,maxDDT = metaPath_Dsim_DT(DD_Sim_sm,train_aff_M,2)
sumDTT,maxDTT = metaPath_DT_Tsim(PP_Sim_sq,train_aff_M,2)

sumDDDT,_ = metaPath_Dsim_DT(sumDDD,train_aff_M,3)
_,maxDDDT = metaPath_Dsim_DT(maxDDD,train_aff_M,3)

sumDTTT,_ = metaPath_DT_Tsim(sumTTT,train_aff_M,3)
_,maxDTTT = metaPath_DT_Tsim(maxTTT,train_aff_M,3)

sumDTDT,maxDTDT = metaPath_DTDT(train_aff_M)
sumDDTT,maxDDTT = metaPath_DDTT(train_aff_M,DD_Sim_sm,PP_Sim_sq)

In [911]:
DT_score = []
lab = []
for i,j in zip(label_row_inds,label_col_inds): #di , #tj      
    pair_scores = (sumDDT[i][j],sumDDDT[i][j],sumDTT[i][j],sumDTTT[i][j], sumDDTT[i][j],\
                   maxDDT[i][j],maxDDDT[i][j],maxDTT[i][j],maxDTTT[i][j],maxDDTT[i][j])
    
    DT_score.append(pair_scores)
    label = aff_exp[i][j]
    lab.append(label)

FV= np.asarray(DT_score)

In [912]:
xg_reg = xgb.XGBRegressor(booster = 'gbtree', objective ='reg:squarederror', eval_metric = 'rmse',
                    colsample_bytree = 0.8, learning_rate = 0.01, max_depth = 15, scale_pos_weight = 1, gamma=0,
                     alpha = 5,n_estimators = 433, tree_method='gpu_hist',min_child_weight =6, 
                    seed=10, gpu_id=1, n_jobs=-1) #better & faster

In [913]:
X = np.array(FV,dtype=np.float32)
# Y = np.array(lab,dtype=np.float32)

min_max_scaler = MinMaxScaler()
X_train = min_max_scaler.fit(X[train_index])
X_train_transform = min_max_scaler.transform(X[train_index])

xg_reg.fit(X_train_transform, Y[train_index])

XGBRegressor(alpha=5, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eval_metric='rmse',
             gamma=0, gpu_id=1, importance_type='gain',
             interaction_constraints='', learning_rate=0.01, max_delta_step=0,
             max_depth=15, min_child_weight=6, missing=nan,
             monotone_constraints='(0,0,0,0,0,0,0,0,0,0)', n_estimators=433,
             n_jobs=-1, num_parallel_tree=1, random_state=10, reg_alpha=5,
             reg_lambda=1, scale_pos_weight=1, seed=10, subsample=1,
             tree_method='gpu_hist', validate_parameters=1, verbosity=None)

In [914]:
#######################Validation SET Evaluation######################### 

X_valid_transform = min_max_scaler.transform(X[valid_index])
predictedY= xg_reg.predict(X_valid_transform)

print("----------------- *EVALUATION using Validation set* -----------------")
# Evaluation Metrics (RMSE, CI, and AUPR)
print('Mean Squared Error: %.3f' % mean_squared_error(Y[valid_index], predictedY))
print('Concordance index: %3f' % get_cindex(Y[valid_index], predictedY))
print('aupr: %.3f' % average_AUPRC(Y[valid_index], predictedY))


----------------- *EVALUATION using Validation set* -----------------
Mean Squared Error: 2.955
Concordance index: 0.667999
aupr: 0.692


In [915]:
########################################## Test SET Evaluation

print("----------------- *Test Set EVALUATION* -----------------")
X_test_transform = min_max_scaler.transform(X[test_index])
# Make predictions using the testing set
predictedY_test= xg_reg.predict(X_test_transform)

# Evaluation Metrics (RMSE, CI, and AUPR)
print('Mean Squared Error: %.3f' % mean_squared_error(Y[test_index], predictedY_test))
print('Concordance index: %3f' % get_cindex(Y[test_index], predictedY_test))
print('aupr: %.3f' % average_AUPRC(Y[test_index], predictedY_test))

----------------- *Test Set EVALUATION* -----------------
Mean Squared Error: 3.036
Concordance index: 0.662360
aupr: 0.673


## 3- Affinity2Vec Hybrid Model

In [916]:
X = np.concatenate((FV,concatenateFV), axis=1)

xg_reg = xgb.XGBRegressor(booster = 'gbtree', objective ='reg:squarederror', eval_metric = 'rmse',
                    colsample_bytree = 0.8, learning_rate = 0.01, max_depth = 11, scale_pos_weight = 1, gamma=0,
                     alpha = 5,n_estimators = 529, tree_method='gpu_hist',min_child_weight =6, 
                    seed=10, gpu_id=1, n_jobs=-1) #better & faster

min_max_scaler = MinMaxScaler()
X_train = min_max_scaler.fit(X[train_index])
X_train_transform = min_max_scaler.transform(X[train_index])

xg_reg.fit(X_train_transform, Y[train_index])

XGBRegressor(alpha=5, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eval_metric='rmse',
             gamma=0, gpu_id=1, importance_type='gain',
             interaction_constraints='', learning_rate=0.01, max_delta_step=0,
             max_depth=11, min_child_weight=6, missing=nan,
             monotone_constraints='(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)',
             n_estimators=529, n_jobs=-1, num_parallel_tree=1, random_state=10,
             reg_alpha=5, reg_lambda=1, scale_pos_weight=1, seed=10,
             subsample=1, tree_method='gpu_hist', validate_parameters=1,
             verbosity=None)

In [917]:
#######################Validation SET Evaluation######################### 

X_valid_transform = min_max_scaler.transform(X[valid_index])
predictedY= xg_reg.predict(X_valid_transform)

print("----------------- *EVALUATION using Validation set* -----------------")
# Evaluation Metrics (RMSE, CI, and AUPR)
print('Mean Squared Error: %.3f' % mean_squared_error(Y[valid_index], predictedY))
print('Concordance index: %3f' % get_cindex(Y[valid_index], predictedY))
print('aupr: %.3f' % average_AUPRC(Y[valid_index], predictedY))

----------------- *EVALUATION using Validation set* -----------------
Mean Squared Error: 2.221
Concordance index: 0.730162
aupr: 0.802


In [918]:
########################################## Test SET Evaluation

print("----------------- *Test Set EVALUATION* -----------------")
X_test_transform = min_max_scaler.transform(X[test_index])
# Make predictions using the testing set
predictedY_test= xg_reg.predict(X_test_transform)

# Evaluation Metrics (RMSE, CI, and AUPR)
print('Mean Squared Error: %.3f' % mean_squared_error(Y[test_index], predictedY_test))
print('Concordance index: %3f' % get_cindex(Y[test_index], predictedY_test))
print('aupr: %.3f' % average_AUPRC(Y[test_index], predictedY_test))

----------------- *Test Set EVALUATION* -----------------
Mean Squared Error: 2.286
Concordance index: 0.722227
aupr: 0.784


##### ----------------------------------- END of the CODE -----------------------------------